# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Hypothesis" data-toc-modified-id="Hypothesis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hypothesis</a></div><div class="lev1 toc-item"><a href="#Analysis" data-toc-modified-id="Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analysis</a></div><div class="lev2 toc-item"><a href="#Imports" data-toc-modified-id="Imports-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Imports</a></div><div class="lev2 toc-item"><a href="#Separation-and-classification" data-toc-modified-id="Separation-and-classification-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Separation and classification</a></div><div class="lev3 toc-item"><a href="#Save-results" data-toc-modified-id="Save-results-221"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Save results</a></div><div class="lev2 toc-item"><a href="#Statistical-analysis" data-toc-modified-id="Statistical-analysis-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Statistical analysis</a></div><div class="lev3 toc-item"><a href="#Two-way-anova" data-toc-modified-id="Two-way-anova-231"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Two-way anova</a></div><div class="lev1 toc-item"><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></div>

# Hypothesis

We propose the dynamics of temporal activity in the mPFC may have contractions or dilations that relate to the behavioral duration of the trial. We planned the following analysis in which to test this hypothesis:


###### Decoding performance comparation between:
- Similar-duration trials
- Trials with many duration

###### Expected results:
- The score will be higher on the set of trials with similar durations


###### Method:
1. **Trial selection**: 
    1. Define $T_{+} = \{t_i \in T_{all}  \quad|\quad i > behaviorThreshold \}$
        - At first, behavior threshold was set to zero, for lack of well defined threshold. <br><br>
        
    2. Separation of trial sets $T_{similar}$ and $T_{bigger}$, according to duration $D(t_i)$
        - $T_{similar} = \{t_i \in T_{+}  \quad|\quad D(t_i) \in [1.5s,3s] \}$
        - $T_{bigger} = \{t_i \in T_{+}  \quad|\quad D(t_i) > 1.5s \}$<br><br><br>
    
2. **Bin selection**: Trial cropping
    - From each set, separate and label bins corresponding to times $\in [0.2s, 1.2s]$  <br><br><br>
    
3. **Classification**
    1. Markov cross-validation of each set using N trials
        - N = $0.9 \times |T_{similar}|$
        - Folds: 30
    2. Performance: cohen's kappa, quadratic weighted<br><br><br>
  
4. **Aggregation** of data from the 4 rats
    - Paired t-test

# Analysis

## Imports

In [9]:
from spikeHelper.loadSpike import Rat
import numpy as np
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.svm import SVC
from sklearn.model_selection import GroupShuffleSplit, cross_val_score
import pandas as pd

## Separation and classification

In [121]:
import pickle
optPar = pickle.load(open('Data/optimalParameters_Mon Oct  2 .pickle','rb')).set_index('Rat')

scores={}
n_splits = 100
for n in [7,8,9,10]:
    scores[(n,'similar')] = []
    scores[(n,'bigger')] = []
    #scores[(n,'similarBig')] = []
    
    # Data selection and cropping: Similar trials
    sim = Rat(n);
    sim.selecTrials({'minDuration':1500, 'maxDuration':2500})
    sim.selecTimes(tmin=200,tmax=1200)
    
    # Data selection and cropping: Different trials
    big = Rat(n);
    big.selecTrials({'minDuration':1500})
    big.selecTimes(tmin=200,tmax=1200)
    
    # Data selection and cropping: Similar big trials
    simBig = Rat(n);
    simBig.selecTrials({'minDuration':2500,'maxDuration':5500})
    simBig.selecTimes(tmin=200,tmax=1200)
    
    # The number of training is defined as 90% of the smaller group
    minTrials = min(len(np.unique(sim.trial)),len(np.unique(big.trial)))#,len(np.unique(simBig.trial)))
    markovCV = GroupShuffleSplit(n_splits=n_splits,train_size=int(0.8*minTrials), test_size=int(0.2*minTrials))
    kappa = make_scorer(cohen_kappa_score, weights='quadratic')
    
    # Fit and score similar
    clf = SVC( C=optPar.loc[n,'C'], gamma = 10**optPar.loc[n,'logGamma'])
    scores[(n,'similar')] = cross_val_score(clf, sim.X,sim.y,cv = markovCV.split(sim.X,sim.y,sim.trial),scoring=kappa)
    scores[(n,'bigger')] = cross_val_score(clf, big.X,big.y,cv = markovCV.split(big.X,big.y,big.trial),scoring=kappa)
    #scores[(n,'similarBig')] = cross_val_score(clf, simBig.X,simBig.y,cv = markovCV.split(simBig.X,simBig.y,simBig.trial))

scores = pd.DataFrame(scores)

### Save results

In [ ]:
# Calculated first with default 'accuracy score'
import h5py
f = h5py.File('fromAllRats.hdf5','w')
f.create_group('decoding')
dset = f.create_dataset('temporalChangesAccuracy',data=scores.values)
dset.attrs.create('columnsTrialType',3*[b'bigger',b'similar'])
dset.attrs.create('columnsRatNumber', np.array([7,7,8,8,9,9,10,10]))

In [128]:
# Then calculated and saved with cohen's kappa
import h5py
f = h5py.File('fromAllRats.hdf5','r+')
dset = f['decoding'].create_dataset('temporalChangesKappa',data=scores.values)
dset.attrs.create('columnsTrialType',3*[b'bigger',b'similar'])
dset.attrs.create('columnsRatNumber', np.array([7,7,8,8,9,9,10,10]))

## Statistical analysis

### Two-way anova

In [83]:
import statsmodels.api as sm
import statsmodels.stats as st
from statsmodels.formula.api import ols

In [126]:
data = scores.melt(var_name=['rat','trialType'])
formula = 'value ~ C(trialType)*C(rat)'
model = ols(formula, data).fit()
aov_table = st.anova.anova_lm(model, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(trialType),0.148292,1.0,83.583741,5.044041e-19
C(rat),2.272639,3.0,426.986510,5.925562e-165
C(trialType):C(rat),0.233070,3.0,43.789413,3.453273e-26
Residual,1.405142,792.0,NaN,NaN


In [136]:
pickle.dump(aov_table,open('../../imagens do diario/temporalChanges.pickle','wb'))

In [127]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  value   R-squared:                       0.654
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     213.7
Date:                Mon, 16 Oct 2017   Prob (F-statistic):          1.19e-177
Time:                        23:16:05   Log-Likelihood:                 1402.6
No. Observations:                 800   AIC:                            -2789.
Df Residuals:                     792   BIC:                            -2752.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                0.1288      0.004     30.581      0.000       0.121       0.137
C(trialType)[T.similar]                  0.0278      0.006      4.667      0.000       0.016       0.039
C(rat)[T.8]                              0.1721      0.006     28.885      0.000       0.160       0.184
C(rat)[T.9]                              0.0849      0.006     14.245      0.000       0.073       0.097
C(rat)[T.10]                             0.0890      0.006     14.936      0.000       0.077       0.101
C(trialType)[T.similar]:C(rat)[T.8]     -0.0534      0.008     -6.341      0.000      -0.070      -0.037
C(trialType)[T.similar]:C(rat)[T.9]      0.0414      0.008      4.916      0.000       0.025       0.058
C(trialType)[T.similar]:C(rat)[T.10]     0.0097      0.008      1.152      0.249      -0.007       0.026
==============================================================================
Omnibus:                       11.084   Durbin-Watson:                   2.067
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               17.881
Skew:                          -0.018   Prob(JB):                     0.000131
Kurtosis:                       3.732   Cond. No.                         12.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""